# Word Cloud - Tweeter

In [5]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = './arial.ttf'


from IPython import display
import time

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder\
    .master('local[*]')\
    .appName('WordCloud')\
    .getOrCreate()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/franciscofoz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def trata_tweets(df):
    words = df \
        .select(f.explode(f.split(f.lower('_c0'), " ")).alias("word")) \
        .withColumn('word', f.regexp_replace('word', r'http\S+', '')) \
        .withColumn('word', f.regexp_replace('word', r'@\w+', '')) \
        .withColumn('word', f.regexp_replace('word', 'rt', '')) \
        .na.replace('', None) \
        .na.drop()
    return words


In [7]:
stops = stopwords.words('portuguese')
stops.append('biblioteconomia')
plt.figure(figsize=(20, 10))

while True:
    try:
        words = spark.read.csv('./csv', encoding='latin1')
        words = trata_tweets(words)
        rows = words.collect()
        all_words = ''
        for row in rows:
            all_words = all_words + ' ' + row['word']

        wordcloud = WordCloud(stopwords=stops,
                              background_color="black",
                              width=1920,
                              height=1080,
                              max_words=100,
                              font_path=font_path
                              ).generate(all_words)

        plt.cla()
        plt.axis('off')
        plt.imshow(wordcloud)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        time.sleep(5)
    except KeyboardInterrupt:
        break

ValueError: Only supported for TrueType fonts

<Figure size 1440x720 with 0 Axes>

In [4]:
import matplotlib.font_manager as fm

font_list = fm.findSystemFonts()
for font in font_list:
    print(font)


/usr/share/fonts/truetype/noto/NotoSerifHebrew-Regular.ttf
/usr/share/fonts/truetype/tlwg/Laksaman.ttf
/usr/share/fonts/truetype/dejavu/DejaVuSerifCondensed-Italic.ttf
/usr/share/fonts/opentype/urw-base35/P052-Bold.otf
/usr/share/fonts/opentype/urw-base35/NimbusRoman-Bold.otf
/usr/share/fonts/truetype/noto/NotoSansHebrew-Regular.ttf
/usr/share/fonts/truetype/noto/NotoSerifGeorgian-Bold.ttf
/usr/share/fonts/truetype/noto/NotoSansThaana-Bold.ttf
/usr/share/fonts/truetype/noto/NotoSansMayanNumerals-Regular.ttf
/usr/share/fonts/truetype/dejavu/DejaVuSerifCondensed.ttf
/usr/share/fonts/truetype/ttf-khmeros-core/KhmerOSsys.ttf
/usr/share/fonts/truetype/noto/NotoSerifTamilSlanted-Regular.ttf
/usr/share/fonts/truetype/malayalam/Dyuthi-Regular.ttf
/usr/share/fonts/truetype/noto/NotoSerifSinhala-Bold.ttf
/usr/share/fonts/truetype/fonts-gujr-extra/padmaa.ttf
/usr/share/fonts/truetype/liberation2/LiberationSans-Regular.ttf
/usr/share/fonts/truetype/noto/NotoSansOldHungarian-Regular.ttf
/usr/share/